In [ ]:
'''检测G2的连通性'''

import networkx as nx
import numpy as np
from scipy.spatial.distance import cdist

def connect_components_until_single(G, marks):
    while len(list(nx.connected_components(G))) > 1:
        G = connect_components_improved(G, marks)
    return G

def connect_components_improved(G, marks):
    components = list(nx.connected_components(G))

    G_new = G.copy()

    for comp1 in components:
        comp1_nodes = list(comp1)
        external_nodes = list(set(G.nodes()) - set(comp1))

        min_distances = []
        for node1 in comp1_nodes:
            distances = cdist([marks[node1]], [marks[node] for node in external_nodes]).flatten()
            closest_node = external_nodes[np.argmin(distances)]
            min_distances.append((node1, closest_node, np.min(distances)))

        min_distances.sort(key=lambda x: x[2])
        closest_connections = min_distances[:2]

        for node1, node2, _ in closest_connections:
            G_new.add_edge(node1, node2)

    return G_new


In [ ]:
import networkx as nx

def filter_paths(G, nodes, secondary_nodes):
    """
    改良G2，保留符合条件的路径。
    
    参数:
    - G: networkx 图对象，表示当前的拓扑图 G2。
    - nodes: 包含所有节点（度为3）的列表。
    - secondary_nodes: 包含每个节点直接连接的三个非节点（二级节点）的列表。

    返回:
    - filtered_paths: 经过筛选的路径列表。
    """
    filtered_paths = []

    # 遍历所有节点对
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            node1 = nodes[i]
            node2 = nodes[j]

            # 找到两个节点之间的一条最短路径
            shortest_path = nx.shortest_path(G, source=node1, target=node2)

            # 路径中的中间节点是否包含第三个或更多节点
            intermediate_nodes = shortest_path[1:-1]
            if any(node in nodes for node in intermediate_nodes):
                continue  # 如果包含第三个或更多节点，跳过此路径

            # 统计路径中经过的二级节点数量
            secondary_node_count = sum(1 for node in intermediate_nodes if node in secondary_nodes)
            
            # 检查路径是否恰好通过两个二级节点
            if secondary_node_count == 2:
                filtered_paths.append(shortest_path)

    return filtered_paths
